<h1 align="center">Advanced Image Processing and Analysis</h1>
<h3 align="center">ECE 4438B/ECE 9022/ECE 9202B/BIOMED 9519B/BIOPHYS 9519B/CAMI 9519B</h3>
<h4 align="center"><a href="mailto:echen29@uwo.ca?subject=Day Day 21: Introduction to non-rigid registration"> Elvis Chen, PhD, LL</a></h4>
<h4 align="center">Day 21, March 25, 2019</h4>

## Announcements

1. **Final Exam**: April 23rd, Tuesday, 19:00, ACEB 1415

### Introduction

In this Jupyter Notebook, we will examine the basic concept of non-rigid, i.e. deformable, registration, starting with thin plate spline.

But why?

#### Rigid registration

Rigid registration is valid with:
* images for non-deformable objects/organs, such as skull, femur, and other bony structures, or
* object/organs exhibiting small deformation
  * e.g. quantitative assessment of articular cartilage degeneration or brain tumor size reduction as a result of chemotherapy or growth resulting from disease progression
  
  
<img src="tumor_brain.PNG" style="width:800px"/>

#### Is rigid registration always sufficient?

**NOT** with organs exhibiting significant deformation

e.g. image guided prostate needle biopsy

* MRI shows tumor but imaging is not real-time
* Ultrasound is real-time, it does not show tumor
* Solution: overlay MRI images on ultrasound images

<img src="prostate_MRI_US.PNG" style="width:800px"/>

In [ ]:
import SimpleITK as sitk
import utilities as util

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from ipywidgets import interact, fixed

OUTPUT_DIR = "output"

In [ ]:
img =  sitk.ReadImage('..\\data\\images\\orangutan.jpg')
plt.imshow(sitk.GetArrayViewFromImage(img))

In [ ]:
rotation2D = sitk.Euler2DTransform()
#rotation2D.SetTranslation((300, 0))
rotation2D.SetAngle(np.pi/9)
moving_resamples = sitk.Resample( img, rotation2D, sitk.sitkLinear, 0.0, img.GetPixelID() )
plt.imshow(sitk.GetArrayViewFromImage(moving_resamples))

In rigid registration:
* straight line segment remains a straight line with the same length,
* planar surface remains planar with the same dimensions,
* appropriate for:
  * bone registration (femur, tibia, pelvis),
  * brain registration (when constrained by skull)

#### Affine registration

In addition to rotation and translation, we can also include scaling and shearing

In [ ]:
affine2d = sitk.AffineTransform(2)
affine2d.Scale([1.5,1.1])

moving_resamples = sitk.Resample( img,affine2d, sitk.sitkLinear, 0.0, img.GetPixelID() )
plt.imshow(sitk.GetArrayViewFromImage(moving_resamples))

# why is it smaller?

In [ ]:
affine2d = sitk.AffineTransform(2)
affine2d.Shear( 0, 1, .2)
moving_resamples = sitk.Resample( img,affine2d, sitk.sitkLinear, 0.0, img.GetPixelID() )
plt.imshow(sitk.GetArrayViewFromImage(moving_resamples))

In affine transformation/registration
* Straight line segment remains a straight line with different length,
* planar surface remains planar with different size,
* in addition to rigid motion correction, affine registration involves **scaling** and **shearing** used for:
  * correction of acquisition parameters (e.g. voxel size) and CT scannery ganty tilt,
  * scale difference in a subject's brain registration

### Non-rigid registration

Also called deformable registration. In addition to the **global** registration, it also compensates for additional local deformation:

* line segment does not remain straight line,
* planar surface does not remain planar,
* Necessary for:
  * tissue deformation evaluation/compensation
  * cross-sectional tissue difference and image atlas construction and analysis
  
  
<img src="ventricle.PNG" style="width:800px"/>

#### but where does tissue deformation come from?
* patient motion, body position, gravity

<img src="breast.PNG" style="width:800px"/>

* clinical intervention, e.g. surgery and biopsy
* changes over time, e.g. growth, shrinkage, or atrophy

### Example: Prostate image registration

Prostate image registration: intra\-operative ultrasound is typically acquired using transrectal ultrasound (TRUS):
![TRUS](http://prostatecancer911.com/wp-content/uploads/2015/11/TRUS.jpg)
* MRI/Ultrasound

<img src="prostate_MRI_US.PNG" style="width:800px"/>

* Ultrasound/Ultrasound due to differnt pressure
<img src="prostate_TRUS.PNG" style="width:800px"/>

### Non-rigid registration transformation

* Straight line segment does not remain straight,
* Planar surface does not remain planar,
* More complex than rigid and affine registration transformation,
* Large number of parameters required,
* In general, any registration technique can be described by 3 components:
  1. Methematical transformation that relates the moving image to the fixed image,
  2. Registration similarity measure (objective function), e.g. matching points or images similarity measures,
    * uses the mathematical transformation,
  3. Numerical Optimizer,
* The difference between rigid body and non-rigid registration is the *transformation* component.

#### Transformations are more complex

* Rigid transformation only has $6$ Degrees-of-Freedom (DoF): three translations $(x,y,z)$ and three rotations,

$
T_{rigid} = \begin{pmatrix} R_{3\times 3} & t_{3\times 1}\\
0 & 1 \end{pmatrix}
$
* Non-rigid transformations:
  * Similarity: isotropic scaling followed by rotation and translation: $7$-DoF
  
  
$
\begin{eqnarray}
T_{scale}(s) & = & \begin{pmatrix} s & 0 & 0 & 0\\
0 & s & 0 & 0\\
0 & 0 & s & 0\\
0 & 0 & 0 & 1\end{pmatrix}\\
T_{similarity}(s,R,t) & = & \begin{pmatrix} R_{3\times 3} & t_{3\times 1}\\
0 & 1 \end{pmatrix} \begin{pmatrix} s & 0 & 0 & 0\\
0 & s & 0 & 0\\
0 & 0 & s & 0\\
0 & 0 & 0 & 1\end{pmatrix}
\end{eqnarray}
$

  * Similarity: anisotropic scaling (scaling factors are potentially different in each of the x-, y-, and z-axis): $9$-DoF
   
$
\begin{eqnarray}
T_{scale}(s_x,s_y,s_z) & = & \begin{pmatrix} s_x & 0 & 0 & 0\\
0 & s_y & 0 & 0\\
0 & 0 & s_z & 0\\
0 & 0 & 0 & 1\end{pmatrix}\\
T_{similarity}(s_x,s_y,s_z,R,t) & = & \begin{pmatrix} R_{3\times 3} & t_{3\times 1}\\
0 & 1 \end{pmatrix} \begin{pmatrix} s_x & 0 & 0 & 0\\
0 & s_y & 0 & 0\\
0 & 0 & s_z & 0\\
0 & 0 & 0 & 1\end{pmatrix}
\end{eqnarray}
$

  
  * Affine: additional shear in each of the x-, y-, and z-axis: $15$-DoF
    * A transformation that slants the shape of an object is called the **shear transformation**. In $3$D, we can shear an object along the x-, y-, and z-axis:
    
$
T_{shear} = \begin{pmatrix} 1 & sh_x^y & sh_x^z & 0 \\
sh_y^x & 1 & sh_z^z & 0 \\
sh_z^x & sh_z^y & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
$
  
Up to Affine transformation, we can represent it as a $4\times 4$ matrix:

$
\begin{eqnarray}
T_{affine}(x,y,z) = \begin{bmatrix}x^{'} \\y^{'} \\z^{'} \\1\end{bmatrix}
& = & T_{shear} \cdot T_{rigid} \cdot T_{scale} \cdot \begin{bmatrix}x \\y \\z \\1\end{bmatrix}\\
& = & \begin{bmatrix} a_{1,1} & a_{1,2} &  a_{1,3} & t_x\\
a_{2,1} &  a_{2,2} &  a_{2,3} & t_y \\ 
a_{3,1} &  a_{3,2} &  a_{3,3} & t_z \\ 
0 & 0 & 0 & 1 \end{bmatrix}
\end{eqnarray}
$

  * Curved: typically Dof: $100$ to $1000$
    * **Quadraic** ($30$ DoF)
    * RHS terms are $x^2$, $y^2$, $z^2$, $x y$, $y z$, $x z$, $x$, $y$, $z$, $1$ ($10$ terms)
    
$
T(x,y,z) = \begin{bmatrix}x^{'} \\y^{'} \\z^{'} \\1\end{bmatrix} = \begin{bmatrix}
r_{0,0}  & \cdots & r_{0,8} & r_{0,9}\\
r_{1,0}  & \cdots & r_{1,8} & r_{1,9}\\
r_{2,0}  & \cdots & r_{2,8} & r_{2,9}\\
0 & \cdots  & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x^2\\y^2\\ \vdots \\ 1
\end{bmatrix}
$

* **Cubic** ($60$ DoF)
* **4^{th} order** ($105$ DoF)
* **5^{th} order** ($168$ DoF)
* number of unknown increases rapidly!
* **The difference between rigid-body/affine and non-rigid registration is in the transformation component!**

### Popular Curved Transformations
* Thin-plate splines
  * Belong to the set of radial-basis functions
* Cubic B-spline
  * Belong to the set of B-splines
* Both are better than the polynomial transformations
  * Polynomial requires too many terms to produces a *well behaved* transformation. 
  * For a detailed discussion, refer to [**Automated Image Registration: Intersubject Validation of Linear and Nonlinear Models**. *Journal of Computer Assisted Tomography*. 22(1), pp 153-165, 1998](http://vr2pk9sx9w.search.serialssolutions.com/?ctx_ver=Z39.88-2004&ctx_enc=info%3Aofi%2Fenc%3AUTF-8&rfr_id=info%3Asid%2Fsummon.serialssolutions.com&rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&rft.genre=article&rft.atitle=Automated+image+registration%3A+II.+Intersubject+validation+of+linear+and+nonlinear+models&rft.jtitle=Journal+of+Computer+Assisted+Tomography&rft.au=Woods%2C+Roger+P&rft.au=Grafton%2C+Scott+T&rft.au=Watson%2C+John+D.+G&rft.au=Sicotte%2C+Nancy+L&rft.date=1998&rft.issn=0363-8715&rft.eissn=1532-3145&rft.volume=22&rft.issue=1&rft.spage=153&rft.epage=165&rft_id=info:doi/10.1097%2F00004728-199801000-00028&rft.externalDBID=n%2Fa&rft.externalDocID=28087338&paramdict=en-UK).

### Non-Rigid Transformation using Thin-Plate Spline (TPS)


<img src="nonRigid.PNG" style="width:400px"/>

Image **A** and Image **B** are the original (rectangular) image. After a non-rigid transformation, B will be deformed. Note that in the non-rigid transformation, the *T* we compute goes from the transformed image back to the untransformed image. We do this because it is difficult to find the inverse of deformable transformations.

### Transformation with matching points using basis functions

Generally speaking, we want to displace an image on a pixel-by-pixel basis:

$
T:(x,y,z) \longrightarrow (x',y',z')
$

$
(x',y',z') = T(x,y,z) = (x,y,z) + u(x,y,z)
$
<img src="deform.PNG" style="width:800px"/>

Instead of using a polynomial, use linear combination of (orthogonal) basis functions, $\theta_{i}$ 

$
T(x,y,z) = \begin{bmatrix}x^{'} \\y^{'} \\z^{'} \\1\end{bmatrix} = \begin{bmatrix}
r_{0,0}  & \cdots & r_{0,n}\\
r_{1,0}  & \cdots & r_{1,n}\\
r_{2,0}  & \cdots & r_{2,n}\\
0 & \cdots  &  1
\end{bmatrix}
\begin{bmatrix}
\theta_{1}(x,y,z)\\ \vdots \\ \theta_{n}(x,y,z)\\  1
\end{bmatrix}
$

In general, we can achieve same level of accuracy using smaller number of coificients $(r_{i,j})$

* Basis functions:
  * Trigonometric basis functions (spectral representation of deformation),
  * wavelet basis
  * see [Ashburner and Friston, Human Brain Mapping, 1999](http://www.fil.ion.ucl.ac.uk/spm/doc/papers/john_nonlinear.pdf).

### Splines Transformation -- Overview

Spline transformation is popular for deformable image registration, often used in entertainment industry

[![test](man2ape.PNG)](https://www.youtube.com/watch?v=4NU9ikjqjC0&t=3s)

In medical imaging, it is also used to registration time-series of deformable organs, such as beating heart in MRI, or prostate under different pressure (TRUS or MRI):

<img src="prostate_deformed_grid.PNG" style="width:800px"/>
[Interactive Deformation Registration of Endorectal Prostate MRI Using ITK Thin Plate Splines](http://www.academicradiology.org/article/S1076-6332%2808%2900569-2/pdf)

### Registration Using splines:

In engineering splines refer to long flexible strips of wood or metal used as templates to form the surface of ships and aircrafts:

<img src="splines_woods.PNG" style="width:800px"/>

#### Mathematical splines

Using mathematical splines, similar concept can be used to model shapes and deformation:
* in mathematics, splines provide a tool used to approximate or interpolate functions from *scattered data*,
* some spline functions are based on mechanics (e.g. bending)

<img src="splines_bending.png" style="width:800px"/>
* In this figure, stick elasticity implies *infinite* displacement support. That is, each data point influences all other points in the domain, including vary far points.
<img src = "squre2sphere.PNG" style="width:800px"/>

#### Splines in non-rigid image registration
 
Non-rigid image registration using spline is a **point-based** registration:
* Two corresponding sets of points are needed in the two images,
* Align points in the sets; other opints (pixels) follow according to the spline transformation function.
 
**How it works**
* Identify a set of point $\Phi_{i}$ in the moving image **A**,
* Identify corresponding set of poing $\Phi_{i}^{'}$ in the fixed image **B**,
* Find a spline transformation **T** that
  * Provides matching at point $\Phi_{i}$ and $\Phi_{i}^{'}$
   
   $
   T(\Phi_{i}) = \Phi_{i}^{'}, 
   $
   for $i = 1, \cdots, n$
  * Interpolates the displacement field such that it
    * produces a smoothly varying displacement field between the points and within the image domain
* Set of points are either
  * Anatomical or geometrical landmarks

## Thin Plate Spline (TPS) Transformation

**Concept**: Most appropriate non-rigid transformation is often the one that matches the landmarks *exactly*, and deforms the rest of the space as little as possible (which leads to smooth deformation)

**Plate Bending Analogy**

The name *thin plate spline* refer to a physical anatogy involving the bending of a thin sheet of metal. Just as the metal has rigidty, the TPS fit resists bending also, implying a penalty involving the smoothness of the fitted surface. 

![test](http://profs.etsmtl.ca/hlombaert/thinplates/thinplates.png)
image courtesy of Jarno Elonen demo,

In the picture above, we can conceptualize bending a thin metal plate by a set of control points (7, red spheres). The thin plate intersect these control points **exactly**, and interpolates everything smoothly.

We thus require a mathematical definition of bending energy: borrowing from solid mechanics, the *squared norm of the displacements (w) second derivatives* is a good measure of the bending energy:

$
E_b = \iint_{R^2}((\frac{\partial^2 z}{\partial x^2})^2 + (\frac{\partial^2 z}{\partial x \partial y})^2 + (\frac{\partial^2 z}{\partial y^2})^2) dx dy
$

That is, in the physical setting, the deflection is in the $z$ direction, orthogonal to the plane. In order to apply this idea to the problem of image transformation, one interprets the lifting of the plate as a displacement of the $x$ or $y$ coordinates within the plane. 

Minimizing this energy leads to the know biharmonic equation (where $w$ is the deflection):

$
\Delta^2 w = (\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2})^2 w = f(x,y)
$

For a single prescribed deflection, the following function satisfies the above equation:

$
w(x,y) = -r^2 \log (r^2), r = \sqrt{x^2+y^2}
$

This is what it looks like in 3D:
<img src="rlogr.png" style="width:600px"/>

Note that:
* $w=0$ at the center of the circle,
* $w=0$ at $r=1$,
* $w$ increases $0 < r < \frac{1}{\sqrt{e}} (~0.607)$
* with $r > \frac{1}{\sqrt{e}}$, $w$ decreases
* $w \rightarrow -\inf$ when $r \rightarrow \inf$

Here is what it looks like in 2D

In [ ]:
x = np.linspace(0,2,1000)
y = np.zeros(1000)
for i in range(2,1000):
    y[i] = -1.0*x[i]*x[i]*np.log(x[i]*x[i])
    
plt.plot(x,y)

### TPS Formulation

Solution of the biharmonic equation for a plate undergoing a number of known deflections $(w)$ at specified points (landmarks)
* use Superposition principle

$
w(x,y) = \sum_{j} w_j r_j^2 \log (r_j^2)
$

where $w_j$ is the *weight* of the basis function contributed by a landmark $j$.
* Model each displacement component of a landmark in the image using the above!

<img src = "squre2sphere.PNG" style="width:600px"/>

<img src = "displacementHtoV.PNG" style="width:600px"/>

The least bent surface is given by the following equation:

$
T(x,y) = a_1 + a_2 x + a_3 y + \sum_{j=1}^n w_{j} U(| P_j - (x,y) |)
$

where $P_j$ denotes the $j^{th}$ landmark, $w_i$ is the weight of the basis function $U$ for $j^{th}$ landmark, and $\ P_j-(x,y) |$ is the distance between the $j^{th}$ landmark (control point) and a position $(x,y)$.

$U$ is the a function based on the distance. In the $2$D case, 

$
\begin{eqnarray}
U(r) & = & r^2 \log(r^2)\\
r^2 & = & x^2 + y^2
\end{eqnarray}
$

#### Affine Transformation component

It should be noted that the first three term of the TPS transformation

$
a_i + a_2 x + a_3 y
$

is actually an affine transformation. It specifies how **globally** the image is transformed. In other word, the coefficient $a_1$ corresponds to the *translational* component of the affine transformation, where as $a_2$ and $2_3$ corresponds to the scaling, rotation, and shear component of the affine transformation. This will become clear once we see an actual example.

#### Deformable Transformation (TPS) component

The summation part of the TPS transformation corresponds to the **local** deformation as a linear combination of some distance function of the control point.

*Question*: what happends when $(x_j,y_j) = (x,y)$?

#### Can be extended to 3D

TPS formulation can be extended to $3$D

$
T(x,y,z) = a_1 + a_2 x + a_3 y + a_4 z + \sum_{j+1}^n w_j U(| (x_j,y_j,z_j) - (x,y,z) |)
$

where

$
U(r) =
\begin{cases}
r^2 \log(r^2) & \quad \text{in 2D}\\
|r|  & \quad \text{or}\\
|r|^3  & \quad \text{in 3D}\\
\end{cases}
$

#### TPS Formulation

* This is a smooth function that can be used for interpolation,
* In $2$D transformation between two images it can be defined by $2$ separate thin-plate spline transformations ($2$ displacement components):

$
\begin{eqnarray}
T(x,y) & = & \begin{pmatrix}t_1(x,y)\\t_2(x,y)\end{pmatrix}\\
t_1 & = & x' = a_{1,1} + a_{2,1} x + a_{3,1} y + \sum_{j=1}^n w_{j,1} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
t_2 & = & y' = a_{1,2} + a_{2,2} x + a_{3,2} y + \sum_{j=1}^n w_{j,2} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
\end{eqnarray}
$

* In $3$D transformation between two volumes it can be defined by $3$ separate thin-plate spline transformations ($3$ displacement components):

$
\begin{eqnarray}
T(x,y,z) & = & \begin{pmatrix}t_1(x,y,z)\\t_2(x,y,z)\\t_3(x,y,z)\end{pmatrix}\\
t_1 & = & x' = a_{1,1} + a_{2,1} x + a_{3,1} y + a_{4,1} z + \sum_{j=1}^n w_{j,1} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
t_2 & = & y' = a_{1,2} + a_{2,2} x + a_{3,2} y + a_{4,2} z + \sum_{j=1}^n w_{j,2} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
t_3 & = & z' = a_{1,3} + a_{2,3} x + a_{3,3} y + a_{4,3} z + \sum_{j=1}^n w_{j,3} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
\end{eqnarray}
$

* Coefficients $a$ characterize the affine part of the transformation
* Coefficients $w$ characterizes the non-affine (deformable-spline) part of the transformation
* **NEED MORE CONSTRAINTS!!!** For $n$ matching points
  * (in $2$D) there are $2 n + 6$ coefficients to be identified with only $2 n$ transformation equations
  * (in $3$D) there are $3 n + 12$ coefficients to be identified with only $3 n$ transformation equations

* (in $2$D) $6$ more equations are required to obtain a unique solution,
* (in $3$D) $12$ more equations are required to obtain a unique solution,
* Add 6/12 equations that guarantee:
  * non-affine coefficients $w$ sum to zero
  
  $
  \sum w_j = 0
  $
  
  ($2$ equations in $2$D, $3$ equations in $3$D)
  * their cross-products with $x$, $y$, and $z$ coordinates of the control points sum to zero:
  
  $
  \begin{eqnarray}
  \sum w_j x_j & = & 0\\
  \sum w_j y_j & = & 0\\
  \sum w_j z_j & = & 0\\
  \end{eqnarray}
  $
  
  ($4$ equations in $2$D, $9$ equations in $3$D)

#### TPS Implementation

Refer to the paper by Fred L. Bookstein

Principal Warps: Thin-Plate Splines and the Decomposition of Deformations

IEEE Transactions on Pattern Analysis and Machine Intelligence 11(6), pp 567, 1989

Algorithm in $2$D:
* First, form two matrices $K$, $P$, and $V$  as follows:

$
\begin{eqnarray}
K & = &\begin{bmatrix} 0 & U(r_{1,2}) & \cdots & U(r_{1,n}) \\
U(r_{2,1}) & 0 & \cdots & U(r_{2,n}) \\
\cdots & \cdots & \cdots & \cdots \\
U(r_{n,1}) & U(r_{n,2}) & \cdots & 0 \\
\end{bmatrix}_{n \times n}\\
P & = & \begin{bmatrix}
1 & x_1 & y_1\\
1 & x_2 & y_2\\
\cdots & \cdots & \cdots \\
1 & x_n & y_n\\
\end{bmatrix}_{n \times 3} \\
V & = & \begin{bmatrix}
1 & x^{'}_1 & y^{'}_1\\
1 & x^{'}_2 & y^{'}_2\\
\cdots & \cdots & \cdots \\
1 & x^{'}_n & y^{'}_n\\
\end{bmatrix}_{n \times 3} \\
\end{eqnarray}
$

where $r_{i,j} = |(x_i,y_i)-(x_j,y_j)|$. $P$ is the set of landmarks in image **A**, and $V$ is the corresponding set of landmarks in image **B**.

* Second, construct another matrix $L$ based on $K$ and $P$:

$
L = \left[
\begin{array}{c|c}
K & P\\
\hline
P^{T} & 0
\end{array}
\right]
$

* Third, form the following equations:
    
$
\left[
\begin{array}{c|c}
K & P\\
\hline
P^{T} & 0
\end{array}
\right] 
\left[ 
\begin{array}{c}
b \\
\hline
a\end{array}
\right] = 
\left[
\begin{array}{c}
V\\
\hline
0
\end{array}
\right]
$


In $2$D:
* $a$ is a $3 \times 2$ matrix ($1^{st}$ column for x-dir, $2^{nd}$ for y-dir)
* $b$ is a $n \times 2$ matrix ($1^{st}$ column for x-dir, $2^{nd}$ for y-dir)

#### Solve for $a$ and $b$

$ 
\left[ 
\begin{array}{c}
b \\
a\end{array}
\right] =  
\left[
\begin{array}{cc}
K & P\\
P^{T} & 0
\end{array}
\right]^{-1}  
\left[
\begin{array}{c}
V\\
0
\end{array}
\right]
$

#### 2D Example

Suppose we have $2$ $2$D images with $4$ control points:

<img src = "TPS_2D_example.PNG" style="width:600px"/>

Define the folowing:

$
\begin{eqnarray}
P_1 & = & \begin{bmatrix}0\\1\end{bmatrix}\\
P_2 & = & \begin{bmatrix}-1\\0\end{bmatrix}\\
P_3 & = & \begin{bmatrix}0\\-1\end{bmatrix}\\
P_4 & = & \begin{bmatrix}1\\0\end{bmatrix}\\
\end{eqnarray}
$

which means

$
\begin{eqnarray}
P & = & \begin{bmatrix}1 & 0 & 1 \\
1 & -1 & 0 \\
1 & 0 & -1 \\
1 & 1 & 0\end{bmatrix}
\end{eqnarray}
$

Given the correspondence, we can define

$
\begin{eqnarray}
V & = & \begin{bmatrix} 0 & 0.75 \\
 -1 & 0.25 \\
 0 & -1.25 \\
 1 & 0.25\end{bmatrix}
\end{eqnarray}
$

To construct $K$, note that we are only dealing with 2 distances:

$
\begin{eqnarray}
U(r_{1,2}) & = & U(\sqrt{2}) = 2 log(2) = 1.3863\\
U(r_{2,3}) & = & U(\sqrt{4}) = 4 log(4) = 5.5452\\
\end{eqnarray}
$

and

$
K = \begin{bmatrix}
0 & 1.3863 & 5.5452& 1.3863\\
1.3863 & 0 & 1.3863 & 5.5452 \\
5.5452 & 1.3863 & 0 & 1.3863\\
1.3863 & 5.5452 & 1.3863 & 0
\end{bmatrix}
$

We can now construct $L$ using $K$ and $P$:

$
L = \left[
\begin{array}{cccc|ccc}
0 & 1.3863 & 5.5452& 1.3863 & 1 & 0 & 1\\
1.3863 & 0 & 1.3863 & 5.5452 & 1 &-1 & 0\\
5.5452 & 1.3863 & 0 & 1.3863 & 1 & 0 & -1\\
1.3863 & 5.5452 & 1.3863 & 0 & 1 & 1 &0\\
\hline
1 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & -1 & 0 & 1 & 0 &0 & 0\\
1 & 0 & -1 & 0 & 0 & 0 & 0
\end{array}
\right]
$

#### Solving for the coefficients

$
\left[
\begin{array}{c}
b \\
\hline
a
\end{array}
\right] = L^{-1} 
\left[
\begin{array}{c}
V \\
\hline
0
\end{array}
\right] =
\left[
\begin{array}{cc}         
0&   -0.0902\\
         0  &  0.0902\\
         0  & -0.0902\\
         0  &  0.0902\\
         \hline
         0  &       0\\
    1.0   &      0\\
         0   & 1.0\\
\end{array}
\right]
$

The first column of the coefficients specify the formula for the $x$-coordinate of the image of $(x,y)$, the second column, those for the $y$-coordinate.

#### Transformed Image Construction

Once $a$ and $b$ parameters are computed, the new position of each pixel $(x,y)$ in the moving image can be determined using:

(in $2$D)

$
\begin{eqnarray}
x' & = & a_{1,1} + a_{2,1} x + a_{3,1} y + \sum_{j=1}^n w_{j,1} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
y' & = & a_{1,2} + a_{2,2} x + a_{3,2} y + \sum_{j=1}^n w_{j,2} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
\end{eqnarray}
$

(in $3$D)

$
\begin{eqnarray}
x' & = & a_{1,1} + a_{2,1} x + a_{3,1} y + a_{4,1} z + \sum_{j=1}^n w_{j,1} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
y' & = & a_{1,2} + a_{2,2} x + a_{3,2} y + a_{4,2} z + \sum_{j=1}^n w_{j,2} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
z' & = & a_{1,3} + a_{2,3} x + a_{3,3} y + a_{4,3} z + \sum_{j=1}^n w_{j,3} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
\end{eqnarray}
$

To obtain image intensity values on the new image grid points, [scattered-data interpolation will be required](https://www.mathworks.com/help/matlab/math/interpolating-scattered-data.html).

## Fun examples with TPS Registration

<img src = "Hawkes2baboon.PNG" style="width:800px"/>

This is David Hawkes from University College London, UK. 

While ITK has an implementation of the [thin plate spline transformation](https://itk.org/Doxygen/html/classitk_1_1ThinPlateSplineKernelTransform.html), SimpleITK has not wrapped this particular function into its Python Interface.  If you are interesting in using ITK directly and am comfortable with C++, you can take a look at an example code in C++ [here](https://itk.org/Doxygen/html/Examples_2RegistrationITKv4_2ThinPlateSplineWarp_8cxx-example.html).

To give you a taste of what TPS can do, we will use an open-source software called [Slicer](https://www.slicer.org/). Python codes is presented below.

First, go to [3DSlicer **download** website](https://download.slicer.org/):
* Download the stable release of the 3DSlicer appropriate for your operating system,
* Install and execute it:

<img src = "3DSlicer.PNG" style="width:800px"/>

* Enable the [python interface](https://www.slicer.org/wiki/Documentation/Nightly/Developers/Python_scripting)
  * Control-3 on windows/linux, Command-3 on mac
  
<img src = "3DSlicer_Python.PNG" style="width:800px"/>

Below are the python code segments to initialize the image processing pipeline within the 3DSlicer framework.

A *renderer* is the class that *draws* on the screen.

We will read a [jpeg image](https://www.vtk.org/doc/nightly/html/classvtkJPEGReader.html) file:
* Other common image reader filters include:
  * [BMP](https://www.vtk.org/doc/nightly/html/classvtkBMPReader.html),
  * [TIFF](https://www.vtk.org/doc/nightly/html/classvtkTIFFReader.html),
  * [PNG](https://www.vtk.org/doc/nightly/html/classvtkPNGReader.html)

```python
renderer = slicer.app.layoutManager().threeDWidget(0).threeDView().renderWindow().GetRenderers().GetFirstRenderer()
reader = vtk.vtkJPEGReader()
reader.SetFileName('C:\\Users\\chene\\Downloads\\temp\\AIP_W18_Notebooks\\data\\images\\monkey.jpg')
reader.Update()
```

The image we read has a pixel count of $800\times 600$. We will use this information to create an evenly-spaced grid for visualization:

```python
imageGrid = vtk.vtkImageGridSource()
imageGrid.SetGridSpacing(50,50,0)                 # create a grid of 50x50 pixels
imageGrid.SetGridOrigin(0,0,0)
imageGrid.SetDataExtent(0,800,0,600,0,0)          # the size of our image, in pixels
imageGrid.SetDataScalarTypeToUnsignedChar()
```

Blend the grid with the image:

```python
table = vtk.vtkLookupTable()
table.SetTableRange(0,1)
table.SetAlphaRange(0,1)
table.SetHueRange(.15,.15)
table.SetSaturationRange(1,1)
table.SetValueRange(0,1)
table.Build()

alpha = vtk.vtkImageMapToColors()
alpha.SetInputConnection( imageGrid.GetOutputPort() )
alpha.SetLookupTable( table )

blend = vtk.vtkImageBlend()
blend.AddInputConnection( 0, reader.GetOutputPort() )  # the image we read
blend.AddInputConnection( 0, alpha.GetOutputPort() )   # image of the grid
```

Now we need to specify a set of landmarks. This information is stored as a set of points:

```python
p1 = vtk.vtkPoints()  # the source points
p2 = vtk.vtkPoints()  # the target points
```

Create a TPS filter

```python
transform = vtk.vtkThinPlateSplineTransform()
transform.SetSourceLandmarks( p1 )
transform.SetTargetLandmarks( p2 )
transform.SetBasisToR2LogR()
transform.Inverse()
```

Connect the image processing pipelines

```python
# create a new image based on the transform
reslice = vtk.vtkImageReslice()
reslice.SetInputConnection( blend.GetOutputPort() )
reslice.SetResliceTransform( transform )
reslice.SetInterpolationModeToLinear()

# visualize it and send it to the renderer
map = vtk.vtkImageMapper()
map.SetInputConnection( reslice.GetOutputPort() )
map.SetColorWindow( 255.0 )
map.SetColorLevel( 127.5 )
map.SetZSlice( 0 )
```

Lastly, specify the control points:

```python
p1.SetNumberOfPoints( 5 )
p1.SetPoint( 0, 324,  imagey-499, 0) # right point on the lip
p1.SetPoint( 1, 378,  imagey-540, 0) # center point on the lip
p1.SetPoint( 2, 451,  imagey-510, 0) # left point on the lip
p1.SetPoint( 3, 343,  imagey-234, 0) # right eye
p1.SetPoint( 4, 452,  imagey-229, 0) # lef eye

p2.SetNumberOfPoints( 5 )
p2.SetPoint( 0, 324,  imagey-540, 0) # right point on the lip
p2.SetPoint( 1, 378,  imagey-540, 0) # center point on the lip
p2.SetPoint( 2, 451,  imagey-540, 0) # left point on the lip
p2.SetPoint( 3, 343,  imagey-234, 0) # right eye
p2.SetPoint( 4, 452,  imagey-229, 0) # left eye

act = vtk.vtkActor2D()
act.SetMapper( map )
act.SetPosition(0,0)
renderer.AddActor( act )
```

Click on the 3DSlicer window and an image will be drawn.

The complete **undocumented** Python code can be found [here](sadTPS.py).

## Exercise

### Final Exam Question:

The following figure shows an axial CT image of a patient who was diagnosed as having Hepatocellular Carcinoma (HCC). The patient's oncologist decided to treat him using radiation therapy. To adjust the tumor radiation dose over the course of treatment, deformable image registration is required. For this purpose, a Medical Physicist suggested using the Thin Plate Spline (TPS) method.

<img src = "HCC.PNG" style="width:600px"/>

* Is the TPS method a good choice in this case? Explain why.
* Select $\approx 10$ points outlining the tumor in addition to $\approx 10$ other points you need to select elsewhere in the image in order to have a matching point set necessary to obtain reasonable registration with TPS method. Show each of the $\approx 20$ points with an $\times$ mark on the image below in question sheet that you will hand in along with your answers.

### TPS Registration Applications

Image Registration
* *With point matching*: [**Registration of images with geometric distortions** by A. Goshtasby, *IEEE Transactions on Geoscience and Remote Sensing*, 26(1):60-64, 1998](http://ieeexplore.ieee.org/document/3000/).
* *With voxel similarity measures*: [**Demonstration of accuracy and clinical versatility of mutual information for automatic multimodality image fusion using affine and thin-plate spline warped geometric deformations** by Meyer et al., *Medical Image Analysis*, 1(3):195-207, 1997](https://www.sciencedirect.com/science/article/pii/S1361841597850104)


#### Advantage:

Control points can have arbitrary spatial distribution:
  * In theory, choose as many as you like,
  * But registration accurady depends on how well these landmarks can be determined, i.e. Fiducial Localization Error (FLE)
  * More landmarks means more parameters to determine,
  
#### Disadvantages:
* Requires point matching, $1:1$ correspondence
  * which requires manual work,
  * and expertise (medical images == medical expertise)
* Control points have global influence since the use of basis function has infinite support!
  * it *cannot* handle local distortion effectively
  * see [**Radial basis functions with compact support for elastic registration of medical images** by Fornefett, Rohr, and Stiehl, *Image and Vision Computing*, 19(1-2):87-96, 2001](https://www.sciencedirect.com/science/article/pii/S0262885600000573)